In [9]:
print('Importing packages...')

import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

from keras.layers import Merge
from keras.layers import TimeDistributed, Lambda
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence, text

import h5py # Used to save models

Importing packages...


In [12]:
print('Reading Data...')

# Output directory
train_loc = 'local' # floyd or local
num_epochs = 1  # Originally used 200

if train_loc == 'floyd':
    output = 'output/'
    data_loc = 'data/'
else:
    output = '../output_conv/'
    data_loc = '../data/'

# Import data
csv_train = data_loc + 'train.csv'
data = pd.read_csv(csv_train)
y = data.is_duplicate.values


# Tokenize text

num_words = 200000  # The total number of word to incorperates in the search
max_len = 40 # The maximum length of a sequence

try:
    import cPickle as pickle # If in python 2 environment
except:
    import pickle # If in a pythnon 3 environment

######################################################
# When training in one go use below
######################################################
tk = text.Tokenizer(num_words=num_words)  # Create a tokenizer object

tk.fit_on_texts(list(data.question1.values) +
                 list(data.question2.values.astype(str)))
pickle.dump(tk,open(output + 'tokenizer.p','wb'))

######################################################
# When using tokenizer made earlier, use below
######################################################

# tk = pickle.load(open('../data/tokenizer.p','rb'))

######################################################

word_index = tk.word_index

x1 = tk.texts_to_sequences(data.question1.values)
x1 = sequence.pad_sequences(x1,maxlen=max_len)

x2 = tk.texts_to_sequences(data.question2.values.astype(str))
x2 = sequence.pad_sequences(x2,maxlen=max_len)

ytrain_enc = np_utils.to_categorical(y)

Reading Data...


KeyboardInterrupt: 

In [ ]:
print('Generating embeddings...')

embeddings_index = {}
f = open(data_loc + 'glove.840B.300d.txt')
unfound = []
unfound_vals = []
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        unfound.append(word)
        unfound_vals.append(values[1:])
f.close()

print('Found %s word vectors.' % len(embeddings_index))

print('Creating embedding matrix...')

embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


max_features = 200000
filter_length = 5
nb_filter = 64
pool_length = 4

In [11]:
print('Build model...')


# Question 1 - Embeddings -> Convolutional

model3 = Sequential()
model3.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))
model3.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))
# model3.add(Dropout(0.2))

# model3.add(Convolution1D(nb_filter=nb_filter,
#                          filter_length=filter_length,
#                          border_mode='valid',
#                          activation='relu',
#                          subsample_length=1))

model3.add(GlobalMaxPooling1D())
# model3.add(Dropout(0.2))

# model3.add(Dense(300))
# model3.add(Dropout(0.2))
# model3.add(BatchNormalization())



# Question 2 - Embeddings -> Convolutional

model4 = Sequential()
model4.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))
model4.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))
# model4.add(Dropout(0.2))

# model4.add(Convolution1D(nb_filter=nb_filter,
#                          filter_length=filter_length,
#                          border_mode='valid',
#                          activation='relu',
#                          subsample_length=1))

model4.add(GlobalMaxPooling1D())
# model4.add(Dropout(0.2))

# model4.add(Dense(300))
# model4.add(Dropout(0.2))
# model4.add(BatchNormalization())

# Merge all models - MERGE ALL QUESTIONS

merged_model = Sequential()
merged_model.add(Merge([model3, model4], mode='concat'))
merged_model.add(BatchNormalization())

# Feed Forward Network
# merged_model.add(Dense(300))
# merged_model.add(PReLU())
# merged_model.add(Dropout(0.2))
# merged_model.add(BatchNormalization())
#
# merged_model.add(Dense(300))
# merged_model.add(PReLU())
# merged_model.add(Dropout(0.2))
# merged_model.add(BatchNormalization())
#
# merged_model.add(Dense(300))
# merged_model.add(PReLU())
# merged_model.add(Dropout(0.2))
# merged_model.add(BatchNormalization())

# Final node gives binary output
merged_model.add(Dense(1))
merged_model.add(Activation('sigmoid'))

Build model...


NameError: name 'word_index' is not defined

In [7]:
# Loss, Optimizer, Accuracy
merged_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 384
# Save checkpoints
checkpoint = ModelCheckpoint(output+'weights.h5', monitor='val_acc', save_best_only=True, verbose=2)

from keras.callbacks import TensorBoard

checkpoint_board = Tensorboard(log_dir=output+'/logs/',batch_size=batch_size)

from keras.callbacks import EarlyStopping

checkpoint_3 = EarlyStopping()

NameError: name 'merged_model' is not defined

In [47]:
# Train model on x1 and x2
history = merged_model.fit([x1, x2], y=y, batch_size=batch_size, nb_epoch=num_epochs,
                 verbose=1, validation_split=0.1, shuffle=True, callbacks=[checkpoint]) # Add tensorboard in callbacks list

pickle.dump(tk,open(output + 'tokenizer.p','wb'))




Train on 363861 samples, validate on 40429 samples
Epoch 1/1
363861/363861 [==============================] - 229s - loss: 0.5295 - acc: 0.7381 - val_loss: 0.4975 - val_acc: 0.7566


In [ ]:
# Save Model

# serialize model to JSON
model_json = merged_model.to_json()
with open(output + "model.json", "w") as json_file:
    json_file.write(model_json)

# LOAD MODEL

In [8]:
from keras.models import model_from_json

# Load Model

# load json and create model
json_file = open(output+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(output+"weights.h5")
print("Loaded model from disk")


# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate([x1,x2], y, verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))


/Users/dattlee/miniconda/envs/quora_question_pairs/lib/python2.7/site-packages/keras/engine/topology.py:1252: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


Loaded model from disk
404192/404290 [============================>.] - ETA: 0sacc: 80.81%


In [11]:
# Import data
csv_test = data_loc + 'test.csv'
test_data = pd.read_csv(csv_test)

# # Tokenize text
# num_words = 200000  # The total number of word to incorperates in the search
# tk = text.Tokenizer(num_words=num_words)  # Create a tokenizer object

# max_len = 40 # The maximum length of a sequence
# tk.fit_on_texts(list(data.question1.values) +
#                 list(data.question2.values.astype(str)))

t_x1 = tk.texts_to_sequences(test_data.question1.values.astype(str))
t_x1 = sequence.pad_sequences(t_x1,maxlen=max_len)

t_x2 = tk.texts_to_sequences(test_data.question2.values.astype(str))
t_x2 = sequence.pad_sequences(t_x2,maxlen=max_len)

In [12]:
p_test = loaded_model.predict([t_x1,t_x2],batch_size=100,verbose=1)

2345600/2345796 [============================>.] - ETA: 0s

In [14]:
# Predict
df_test = pd.read_csv(data_loc+'test.csv')

sub = pd.DataFrame()
sub['test_id'] = df_test['test_id']
sub['is_duplicate'] = p_test
sub.to_csv(output + 'shitty_conv.csv', index=False)